In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# authentication
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


In [3]:
# importing specific components needed from ADK and the generative AI library
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

print("ADK Components imported successfully")

ADK Components imported successfully


In [4]:
# helper function only for notebook runners
# used throughtout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers

def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [5]:
# configure retry option
# working with llm can transient errors like rate limits or temporary service unavailability
# retry option will automaticlly handle these failure by retrying the request

retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

what is AI Agent

LLM :: Prompt -> LLM -> Text

A step further is AI Agent

AI Agent :: Prompt -> Agent -> Thought -> Action -> Observation -> Final Answer


# Agent for making google search

setting agent properties >> what to do and how to do

main properties

- name and description : to identify the agent
- model : sepecific llm 'gemini-2.5-flash-lite'
- instruction : guideing prompt for agent >> tells agent what its goal is and how to behave
- tools : list of tools agent can use

>> Start :: google_search tool for finding upto date information online

LlmAgent() = special class for LLM-based agents. >> used in documentation https://google.github.io/adk-docs/agents/llm-agents/#defining-the-agents-identity-and-purpose

Agent() = more general agent class (what Kaggle prefers)

>> kaggle have build in tools there are many. we can also make custom tools and wrap them using @tools then function definition in kaggle


In [6]:
# root agent is top agent which manages other sub agents
root_agent = Agent(
    name="helpful_assistant",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="A agent that can answer complex questions.",
    instruction="You are a helpful assistant. Use google search for answering.",
    tools=[google_search],
)
print("Root Agent defined")
# description="A simple agent that can answer general questions.",
#     instruction="You are a helpful assistant. Use Google Search for current info or if unsure.",

Root Agent defined


### Running agent
sending query to agent
- for doing this we need runner
- central component with in ADK acts as orchestrator

>> it manages our conversation, sends our messages to the agent and handles responses

In [7]:
# Create an InMemoryRunner and tell it to use our root_agent
runner = InMemoryRunner(agent=root_agent)
print("Runner created")
# we are using the Python Runner directly in this notebook.
# We can also run agents using ADK command-line tools 
# such as adk run, adk web, or adk api_server >> check doc for more

Runner created


In [8]:
# Now lets call the .run_debug() method to send our prompt and get an answer
response = await runner.run_debug(
    "what are the latest treands going in technology these days"
)


 ### Created new session: debug_session_id

User > what are the latest treands going in technology these days
helpful_assistant > As of 2025, several key technology trends are shaping the future, with Artificial Intelligence (AI) being a dominant force across many sectors.

Here are some of the most significant trends:

*   **Generative AI:** This form of AI is capable of creating diverse content, including text, images, and music, from simple prompts. Its impact is profound, transforming creative fields and enhancing content creation processes. Advancements in models like GPT-4o are enabling new applications in marketing, entertainment, and education.
*   **Agentic AI:** This trend focuses on AI systems that can plan and take independent action to achieve user-defined goals. These "virtual coworkers" can autonomously execute multi-step workflows, leading to improved efficiency in various industries.
*   **AI and Machine Learning (ML) Advancements:** AI and ML are becoming more sophis

Agent performed a Google search to get the information

1. The agent is aware and it inspects which tool ih has available to use.
2. The agent was specified about the use of search tool and to get the information or if unsure of the answer

# ADK web UI

In [9]:
!adk create sample-agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY


Agent created in /kaggle/working/sample-agent:
- .env
- __init__.py
- agent.py



In [10]:
# getting url to access the ADK web UI in Kaggle
url_prefix = get_adk_proxy_url()

Exception: No running Jupyter servers found.

In [ ]:
!adk web --url_prefix {url_prefix}